In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
import json

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_SHEETS_CREDENTIALS"] = userdata.get('GOOGLE_SHEETS_CREDENTIALS')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Importa Google Planilhas
!pip install gspread

import gspread
from google.oauth2.service_account import Credentials

from google.adk.tools import BaseTool
from pydantic import Field, BaseModel
import gspread
from google.oauth2.service_account import Credentials

# Define o escopo necessário para acessar o Google Sheets
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

In [ ]:
# Carrega as credenciais do arquivo JSON (substitua pelo caminho do seu arquivo)
# Certifique-se de que você tem um arquivo de credenciais de conta de serviço e que a API do Google Sheets e Drive estão habilitadas
# Este arquivo deve ser gerado no Google Cloud Platform
# Recomenda-se armazenar este caminho ou o conteúdo do arquivo de forma segura, por exemplo, usando o Secret Manager ou Google Colab userdata
try:
    # Tentativa de carregar credenciais do Colab userdata se aplicável
    # Substitua 'GOOGLE_SHEETS_CREDENTIALS' pelo nome da sua chave no userdata
    creds_data_str = userdata.get('GOOGLE_SHEETS_CREDENTIALS') # Obtenha os dados da string
    # Parse a string JSON em um dicionário
    creds_data = json.loads(creds_data_str)
    creds = Credentials.from_service_account_info(creds_data, scopes=SCOPES)
except Exception as e:
    print(f"Erro ao carregar credenciais do userdata: {e}")
    print("Por favor, verifique se a chave 'GOOGLE_SHEETS_CREDENTIALS' está configurada corretamente no Google Colab userdata.")
    print("Alternativamente, você pode carregar as credenciais de um arquivo JSON no seu ambiente.")
    # Como alternativa, você pode carregar de um arquivo JSON local se não estiver usando userdata
    # creds = Credentials.from_service_account_file('path/to/your/credentials.json', scopes=SCOPES)
    creds = None # Define creds como None para evitar erros posteriores se o carregamento falhar


In [ ]:
# Autentica e cria o cliente gspread
if creds:
    client_gspread = gspread.authorize(creds)
else:
    client_gspread = None
    print("Cliente gspread não autenticado. As ferramentas que dependem dele não funcionarão.")

In [ ]:
# Define o modelo de entrada para a função 'criar_planilha'
class CriarPlanilhaInput(BaseModel):
    """Input para a ferramenta que cria uma nova planilha Google."""
    titulo: str = Field(description="O título da nova planilha Google.")

In [ ]:
# Define o modelo de entrada para a função 'escrever_dados'
class EscreverDadosInput(BaseModel):
    """Input para a ferramenta que escreve dados em uma planilha Google."""
    titulo_planilha: str = Field(description="O título da planilha onde os dados serão escritos.")
    nome_aba: str = Field(description="O nome da aba (sheet) onde os dados serão escritos (ex: 'Sheet1').")
    dados: list[list[str]] = Field(description="Uma lista de listas representando as linhas e colunas dos dados a serem escritos.")
    intervalo: str = Field(description="O intervalo onde os dados serão escritos (ex: 'A1').")


class GoogleSheetsTool(BaseTool):
    """
    Uma ferramenta para interagir com o Google Sheets usando gspread.
    Permite criar planilhas e escrever dados.
    """
    name: str = "google_sheets_tool"
    description: str = "Ferramenta para criar e manipular planilhas Google Sheets."

    def __init__(self, client_gspread: gspread.Client):
        super().__init__(name=self.name, description=self.description)
        self.client = client_gspread

    def criar_planilha(self, input: CriarPlanilhaInput):
        """Cria uma nova planilha Google com o título especificado."""
        if not self.client:
            return {"error": "Cliente gspread não autenticado."}
        try:
            planilha = self.client.create(input.titulo)
            return {"sheet_url": planilha.url, "sheet_id": planilha.id}
        except Exception as e:
            return {"error": f"Erro ao criar planilha: {e}"}

    def escrever_dados(self, input: EscreverDadosInput):
        """Escreve dados em uma aba específica de uma planilha Google."""
        if not self.client:
            return {"error": "Cliente gspread não autenticado."}
        try:
            planilha = self.client.open(input.titulo_planilha)
            aba = planilha.worksheet(input.nome_aba)
            aba.update(input.intervalo, input.dados)
            return {"status": "Dados escritos com sucesso."}
        except Exception as e:
            return {"error": f"Erro ao escrever dados na planilha: {e}"}

    def get_function_definitions(self):
        return [
            self.get_function_definition_from_method(self.criar_planilha, CriarPlanilhaInput),
            self.get_function_definition_from_method(self.escrever_dados, EscreverDadosInput),
            # Adicione definições para outras funções (como criar gráficos) aqui
        ]

In [ ]:
# Instancia a ferramenta Google Sheets se o cliente gspread for válido
if client_gspread:
    google_sheets_tool_instance = GoogleSheetsTool(client_gspread)
else:
    google_sheets_tool_instance = None
    print("Instância da ferramenta GoogleSheetsTool não criada devido à falha na autenticação.")


In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Define o modelo de entrada para a função 'criar_grafico'
class CriarGraficoInput(BaseModel):
    """Input para a ferramenta que cria um gráfico em uma planilha Google."""
    titulo_planilha: str = Field(description="O título da planilha onde o gráfico será criado.")
    nome_aba: str = Field(description="O nome da aba (sheet) onde o gráfico será criado.")
    intervalo_dados: str = Field(description="O intervalo dos dados a serem plotados (ex: 'A1:B5').")
    titulo_grafico: str = Field(description="O título do gráfico.")
    tipo_grafico: str = Field(description="O tipo de gráfico (ex: 'LINE' para gráfico de linha).") # Podemos adicionar mais tipos depois
    intervalo_posicao: str = Field(description="O intervalo onde o gráfico será posicionado (ex: 'D1').")


class GoogleSheetsTool(BaseTool):
    """
    Uma ferramenta para interagir com o Google Sheets usando gspread.
    Permite criar planilhas, escrever dados e criar gráficos.
    """
    name: str = "google_sheets_tool"
    description: str = "Ferramenta para criar e manipular planilhas Google Sheets, incluindo criação de gráficos."

    def __init__(self, client_gspread: gspread.Client):
        super().__init__(name=self.name, description=self.description)
        self.client = client_gspread

    def criar_planilha(self, input: CriarPlanilhaInput):
        """Cria uma nova planilha Google com o título especificado."""
        if not self.client:
            return {"error": "Cliente gspread não autenticado."}
        try:
            planilha = self.client.create(input.titulo)
            return {"sheet_url": planilha.url, "sheet_id": planilha.id}
        except Exception as e:
            return {"error": f"Erro ao criar planilha: {e}"}

    def escrever_dados(self, input: EscreverDadosInput):
        """Escreve dados em uma aba específica de uma planilha Google."""
        if not self.client:
            return {"error": "Cliente gspread não autenticado."}
        try:
            planilha = self.client.open(input.titulo_planilha)
            aba = planilha.worksheet(input.nome_aba)
            aba.update(input.intervalo, input.dados)
            return {"status": "Dados escritos com sucesso."}
        except Exception as e:
            return {"error": f"Erro ao escrever dados na planilha: {e}"}

    def criar_grafico(self, input: CriarGraficoInput):
        """Cria um gráfico em uma aba específica de uma planilha Google."""
        if not self.client:
            return {"error": "Cliente gspread não autenticado."}
        try:
            planilha = self.client.open(input.titulo_planilha)
            aba = planilha.worksheet(input.nome_aba)


            # A API do Google Sheets é um pouco verbosa para criar gráficos diretamente via gspread.
            # Precisamos construir a solicitação de adição de gráfico.
            # Este é um exemplo simplificado para um gráfico de linha.
            # Para outros tipos de gráfico e configurações, a estrutura da solicitação muda.

            requests_body = {
                'addChart': {
                    'chart': {
                        'spec': {
                            'title': input.titulo_grafico,
                            'basicChart': {
                                'chartType': input.tipo_grafico, # Ex: 'LINE'
                                'legendPosition': 'BOTTOM_LEGEND',
                                'domains': [
                                    {
                                        'domain': {
                                            'sourceRange': {
                                                'sources': [
                                                    {
                                                        'sheetId': aba.id,
                                                        'range': f"{aba.title}!{input.intervalo_dados.split(':')[0]}:{input.intervalo_dados.split(':')[0]}" # Eixo X
                                                    }
                                                ]
                                            }
                                        }
                                    }
                                ],
                                'series': [
                                    {
                                        'series': {
                                            'sourceRange': {
                                                'sources': [
                                                    {
                                                        'sheetId': aba.id,
                                                        'range': f"{aba.title}!{input.intervalo_dados.split(':')[1]}:{input.intervalo_dados.split(':')[1]}" # Eixo Y (simples, assumindo 2 colunas)
                                                    }
                                                ]
                                            }
                                        },
                                        'targetAxis': 'LEFT_AXIS'
                                    }
                                ],
                                'headerCount': 1 # Assumindo que a primeira linha é cabeçalho
                            }
                        },
                        'position': {
                            'overlayPosition': {
                                'anchorCell': {
                                    'sheetId': aba.id,
                                    'rowIndex': gspread.utils.a1_to_rowcol(input.intervalo_posicao)[0] - 1,
                                    'columnIndex': gspread.utils.a1_to_rowcol(input.intervalo_posicao)[1] - 1,
                                },
                                'offsetXPixels': 5,
                                'offsetYPixels': 5,
                                'widthPixels': 600,
                                'heightPixels': 400
                            }
                        }
                    }
                }
            }

            # Para executar a solicitação, precisamos acessar o cliente subjacente do gspread
            # e usar o método batch_update.

            self.client.batch_update(planilha.id, requests_body)

            return {"status": f"Gráfico '{input.titulo_grafico}' criado com sucesso na aba '{input.nome_aba}'."}
        except Exception as e:
            return {"error": f"Erro ao criar gráfico: {e}"}

    def get_function_definitions(self):
        return [
            self.get_function_definition_from_method(self.criar_planilha, CriarPlanilhaInput),
            self.get_function_definition_from_method(self.escrever_dados, EscreverDadosInput),
            self.get_function_definition_from_method(self.criar_grafico, CriarGraficoInput), # Adicione a nova função aqui
        ]

In [ ]:
##########################################
# --- Agente 1: Buscador de Receitas --- #
##########################################
def agente_buscador(ingredientes):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search) e o youtube
        para buscar receitas que contenham os ingredientes informados.
        Foque em no máximo 10 receitas mais compativeis com os ingredientes.
        Se uma receita tiver somente um dos ingredientes, é possível que ele não seja tão relevante assim
        e pode ser substituído por outra que tenha mais.
        Essas receitas devem ter lista de ingredientes e modo de preparo.
        """,
        description="Agente que busca receitas no Google e Youtube",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Ingredientes: {ingredientes}"

    receitas = call_agent(buscador, entrada_do_agente_buscador)
    return receitas

In [ ]:
################################################
# --- Agente 2: Especialista Gastronômico --- #
################################################
def agente_gastronomico(ingredientes, receitas_buscadas):
    gastronomico = Agent(
        name="agente_gastronomico",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em gastronomia. Com base na lista de receitas com os ingredientes, você deve:
        Montar uma lista com os receitas e classifica-las de acordo com a compatibilidade com os ingredientes.
        Escolher as 5 melhores receitas e retornar elas com seus ingredientes e modo de preparo.
        Usar a ferramenta do Google Search (google_search) e do youtube para buscar mais informações sobre as receitas e videos ensinando como preprar.
        Ao final, você irá montar uma lista dessas receitas com seus ingredientes, modo de prepraro, tempo de preparo, rendimento e até 2 videos cada.
        """,
        description="Agente que escolhe as melhores receitas",
        tools=[google_search]
    )

    entrada_do_agente_gastronomico = f"Ingredientes:{ingredientes}\nReceitas buscadas: {receitas_buscadas}"
    # Executa o agente
    melhores_receitas = call_agent(gastronomico, entrada_do_agente_gastronomico)
    return melhores_receitas

In [ ]:
######################################
# --- Agente 3: Nutricionista --- #
######################################
def agente_nutricionista(ingredientes, melhores_receitas):
  # Inclui a nova ferramenta se ela foi instanciada com sucesso
    tools_list = [google_search]
    if google_sheets_tool_instance:
        tools_list.append(google_sheets_tool_instance)
    else:
        print("Ferramenta Google Sheets não disponível para o agente nutricionista.")

    nutricionista = Agent(
        name="agente_nutricionista",
        model="gemini-2.0-flash",
        instruction="""
            Você é um nutricionista especializado em receitas saudáveis e equilibradas.
            Com base na lista de ingredientes e receitas selecionadas, você deve:
            Avaliar a qualidade nutricional de cada receita, considerando os macronutrientes, micronutrientes e possíveis restrições alimentares.
            Sugerir ajustes nos ingredientes ou no modo de preparo para tornar as receitas mais saudáveis, se necessário.
            Ao final, você irá montar uma lista dessas receitas com seus ingredientes, modo de prepraro, tempo de preparo, rendimento e até 2 videos cada.
            **Usando a ferramenta google_sheets_tool e chamando a função criar_grafico você DEVE criar uma nova planilha Google com o título 'Comparativo Nutricional de Receitas'.**
            **Nesta planilha, na 'Sheet1', escreva uma tabela com as informações nutricionais principais de cada receita (por exemplo, Calorias, Proteínas, Carboidratos, Gorduras) usando a ferramenta google_sheets_tool e especifique o intervalo onde escrever.**
            """,
        description="Agente nutricionista que analisa receitas e cria relatórios e gráficos em Google Sheets.",
        tools=tools_list # Usa a lista de ferramentas com ou sem a ferramenta Google Sheets
    )
    entrada_do_agente_nutricionista = f"Ingredientes: {ingredientes}\nMelhores receitas: {melhores_receitas}"
    # Executa o agente
    refeicoes = call_agent(nutricionista, entrada_do_agente_nutricionista)
    return refeicoes

In [ ]:
##########################################
# --- Agente 4: Redator --- #
##########################################
def agente_redator(ingredientes, refeicoes_selecionadas):
  # Inclui a nova ferramenta se ela foi instanciada com sucesso
    tools_list = [google_search]
    if google_sheets_tool_instance:
        tools_list.append(google_sheets_tool_instance)
    else:
        print("Ferramenta Google Sheets não disponível para o agente redator.")

    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um redator e escritor focado em revistas gastronomicas, a partir da lista de refeições selecionadas, você deve:
            Montar uma lista com as refeições, uma breve história, seus ingredientes, modo de preparo, tempo de preparo, rendimento e até 2 videos cada.
            Também devera criar tabelas com as informações nutricionais de cada refeição e graficos de linhas comparando as refeições utilizanda a ferramenta (google_sheets_tool) e chamando a função (criar_grafico).
            **Em seguida, usando a ferramenta google_sheets_tool e chamando a funçaõ criar_grafico, junto dos dados que você escreveu, crie um GRÁFICO DE LINHA comparando a quantidade de Calorias entre as receitas. Posicione o gráfico na mesma aba, em um intervalo livre como 'D1'. O título do gráfico deve ser 'Comparativo de Calorias'.**
            E por fim deve revisar a ortografia e gramática do texto.
            """,
        description="Agente redator.",
        tools=tools_list # Usa a lista de ferramentas com ou sem a ferramenta Google Sheets]
    )
    entrada_do_agente_redator = f"Ingredientes: {ingredientes}\nRascunho: {refeicoes_selecionadas}"
    # Executa o agente
    receitas_finais = call_agent(redator, refeicoes_selecionadas)
    return receitas_finais

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🍕😋 Vamos procurar a melhor receita para você? 😋🥗")

# --- Obter o Ingredientes do Usuário ---
ingredientes = input("❓ Por favor, digite quais ingrediente você possui para cozinhar: ")

# Inserir lógica do sistema de agentes ################################################
if not ingredientes:
    print("Você esqueceu de digitar os ingredientes!")
else:
    print(f"Maravilha! Vamos então procurar as melhores receitas com {ingredientes}")

    receitas_buscadas = agente_buscador(ingredientes)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Receitas) ---\n")
    display(to_markdown(receitas_buscadas))
    print("--------------------------------------------------------------")

    melhores_receitas = agente_gastronomico(ingredientes, receitas_buscadas)
    print("\n--- 📝 Resultado do Agente 2 (Chefe Gastronômico) ---\n")
    display(to_markdown(melhores_receitas))
    print("--------------------------------------------------------------")

    # Passa a instância da ferramenta se ela existir
    refeicoes_selecionadas = agente_nutricionista(ingredientes, melhores_receitas)
    print("\n--- 📝 Resultado do Agente 3 (Nutricionista) ---\n")
    display(to_markdown(refeicoes_selecionadas))
    print("--------------------------------------------------------------")

    # Passa a instância da ferramenta se ela existir
    receitas_finais = agente_redator(ingredientes, refeicoes_selecionadas)
    print("\n--- 📝 Resultado do Agente 4 (Redator) ---\n")
    display(to_markdown(receitas_finais))
    print("--------------------------------------------------------------")